In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.datasets import load_boston
X,y = load_boston(return_X_y=True)

In [27]:
class DecisionTree():
    def __init__(self, X, y, n_trees, sample_sz, min_leaf):
        self.X,self.y,self.n_trees,self.sample_sz,self.min_leaf=X,y, n_trees, sample_sz, min_leaf
        self.lol=[]
    @staticmethod
    def sse(samples):
        return np.var(samples)*len(samples)
    
    def make_a_split(self,sampleX,sampleY,min_leaf,sample_sz):
        cutoffs=list()
        for feature in sampleX.T:   
            unique_cutoffs=np.unique(feature)[:-1]
            cutoffs.append(np.random.choice(unique_cutoffs,replace=False,size=min(sample_sz,len(unique_cutoffs))))
        variances = np.ones((sampleX.shape[1],sample_sz))*self.sse(sampleY)
        for feature,cutoff_array in enumerate(cutoffs):
            for sample,cutoff in enumerate(cutoff_array):
                variances[feature,sample]=np.nansum([self.sse(sampleY[sampleX[:,feature]<=cutoff]),self.sse(sampleY[sampleX[:,feature]>cutoff])])
                np.nansum([self.sse(sampleY[sampleX[:,feature]<=cutoff]),self.sse(sampleY[sampleX[:,feature]>cutoff])])
        feature,cutoff_num = np.unravel_index(variances.argmin(), variances.shape)
        cutoff_point = cutoffs[feature][cutoff_num]
        f_c = [feature,cutoff_point]
        index1=sampleX[:,feature]>cutoff_point
        index2=sampleX[:,feature]<=cutoff_point
        X1,X2,y1,y2 = sampleX[index1,:],sampleX[index2,:],sampleY[index1],sampleY[index2]
        return X1,y1,X2,y2,f_c
    def split_recursive(self,X,y,sample_sz,min_leaf,n_trees,coords=[],list_of_cutoffs=[]):
        X1,y1,X2,y2,f_c=self.make_a_split(X,y,min_leaf,sample_sz)
        if len(y1)>min_leaf and len(list_of_cutoffs)<(n_trees-1):
            self.split_recursive(X1,y1,sample_sz,min_leaf,n_trees,coords=coords+['a'],list_of_cutoffs=list_of_cutoffs+[f_c])
        else:
            self.lol.append([list_of_cutoffs+[f_c],coords+['a'],y1.mean()])
        if len(y2)>min_leaf and len(list_of_cutoffs)<(n_trees-1):
            self.split_recursive(X2,y2,sample_sz,min_leaf,n_trees,coords=coords+['b'],list_of_cutoffs=list_of_cutoffs+[f_c])
        else:
            self.lol.append([list_of_cutoffs+[f_c],coords+['b'],y2.mean()])
    def fit(self):
        print(self.X)
        self.split_recursive(self,X,self.y,self.sample_sz,self.min_leaf,self.n_trees)
    def predict(self,x2check):
        for i in self.lol:
            stop=False
            for n,pair in enumerate(i[0]):
                if stop==False:
                    if (x2check[pair[0]]>pair[1] and i[1][n]=="a") or (x2check[pair[0]]<=pair[1] and i[1][n]=="b") :
                        if n==len(i[0])-1:
                            return (i[2])
                    else:
                        stop = True



dt = DecisionTree(X,y,20,100,3)
dt.split_recursive(X,y,20,100,3)

dt.predict(X[13])

22.693442622950823

In [36]:
class DecisionTree():
    def __init__(self):
#         self.n_trees,self.sample_sz,self.min_leaf= n_trees, sample_sz, min_leaf
        self.tree=[]
    @staticmethod
    def sse(samples):
        return np.var(samples)*len(samples)
    
    def make_a_split(self,sampleX,sampleY,min_leaf,sample_sz):
        cutoffs=list()
        # make a list of cutoffs using "sample_sz" unique values for each feature
        for feature in sampleX.T:   
            unique_cutoffs=np.unique(feature)[:-1]
            cutoffs.append(np.random.choice(unique_cutoffs,replace=False,size=min(sample_sz,len(unique_cutoffs))))
        # calculate the sse resulting from each split
        variances = np.ones((sampleX.shape[1],sample_sz))*self.sse(sampleY)
        for feature,cutoff_array in enumerate(cutoffs):
            for sample,cutoff in enumerate(cutoff_array):
                variances[feature,sample]=np.nansum([self.sse(sampleY[sampleX[:,feature]<=cutoff]),self.sse(sampleY[sampleX[:,feature]>cutoff])])
                np.nansum([self.sse(sampleY[sampleX[:,feature]<=cutoff]),self.sse(sampleY[sampleX[:,feature]>cutoff])])
        # choose the feature and threshold that minimize the variance
        feature,cutoff_num = np.unravel_index(variances.argmin(), variances.shape)
        cutoff_point = cutoffs[feature][cutoff_num]
        f_c = [feature,cutoff_point]
        index1=sampleX[:,feature]>cutoff_point
        index2=sampleX[:,feature]<=cutoff_point
        X1,X2,y1,y2 = sampleX[index1,:],sampleX[index2,:],sampleY[index1],sampleY[index2]
        return X1,y1,X2,y2,f_c
    def split_recursive(self,X,y,sample_sz,min_leaf,n_trees,coords=[],list_of_cutoffs=[]):
        # split recursively and add the feature and cutoff for each split, along with 'a' for right and 'b' for left
        X1,y1,X2,y2,f_c=self.make_a_split(X,y,min_leaf,sample_sz)
# keep splitting as these conditions are met
        if len(y1)>min_leaf and len(list_of_cutoffs)<(n_trees-1):
            self.split_recursive(X1,y1,sample_sz,min_leaf,n_trees,coords=coords+['a'],list_of_cutoffs=list_of_cutoffs+[f_c])
        else:
# when we stop splitting, add the cutoffs, list of coordinates (right and left) and the mean of the leaf into the tree
            self.tree.append([list_of_cutoffs+[f_c],coords+['a'],y1.mean()])
# do the same for other side of the split
        if len(y2)>min_leaf and len(list_of_cutoffs)<(n_trees-1):
            self.split_recursive(X2,y2,sample_sz,min_leaf,n_trees,coords=coords+['b'],list_of_cutoffs=list_of_cutoffs+[f_c])
        else:
            self.tree.append([list_of_cutoffs+[f_c],coords+['b'],y2.mean()])
    def predict(self,x2check):
        for leaf in self.tree:
            stop=False
            for n,pair in enumerate(leaf[0]):
                if stop==False:
                    # go through each node in the leaf, if it matches continue if not try the next leaf
                    if (x2check[pair[0]]>pair[1] and leaf[1][n]=="a") or (x2check[pair[0]]<=pair[1] and leaf[1][n]=="b") :
                        # if we have gotten to the end of the leaf and it is all correct, predict the leaf mean
                        if n==len(leaf[0])-1:
                            return (leaf[2])
                    else:
                        stop = True



dt = DecisionTree()
dt.split_recursive(X,y,20,100,3)

dt.predict(X[13])

24.050943396226415